In [1]:
import ollama
import datasets
import pandas as pd
import re

# Подготовка заданного числа вопросов и ответов

In [2]:
from datasets import load_dataset

dataset = load_dataset("gsm8k", 'main')

In [3]:
full_answers = (dataset['train'][:]['answer'][:100])

In [4]:
def get_short_answers(full_answer):
    return re.search('(?s)(?<=#### )(.*$)', full_answer)[0]

In [5]:
short_answers = list(map(get_short_answers, full_answers))

In [6]:
questions = dataset['train'][:]['question'][:100]

# Прогон модели 15 раз:

In [10]:
def ask_question(question):
    x = ollama.chat(model='mistral', messages=[
    {
    'role': 'user',
    'content': question,
    }
    ])
    return x['message']['content']

In [8]:
# запятая точ ка после трех,
# если одна то проверять .\, пробел и доллар
def check_answer(row):
    # добавить точки и запятые

    end_of_mistral_answer = row.mistral_answers[-200:]
    answer_length = len(row.true_answers)
    match = re.search(r'\D' + str(row.true_answers) + r'\D', end_of_mistral_answer)
    row.check = True if match else False
    # для случаев 4,351
    if answer_length > 3:
        if row.true_answers in end_of_mistral_answer or row.true_answers[:-3] + ',' + row.true_answers[-3:] in end_of_mistral_answer or row.true_answers[:-3] + '.' + row.true_answers[-3:] in end_of_mistral_answer:
            row.check = True
        else:
            row.check = False
    '''else:
        # ес
        if  row.true_answers in end_of_mistral_answer[-10:]
        if ' ' + row.true_answers + ' ' in end_of_mistral_answer or '$' + row.true_answers + ' ' in end_of_mistral_answer or ' ' + row.true_answers + '$' in end_of_mistral_answer:
            row.check = True
        else:
            row.check = False'''
    return row
# !!!!!!!!!!!!!!!!!!!
# ЕСТЬ РИСК ТОГО, ЧТО ОТВЕТ ВСТРЕЧАЕТСЯ В НАЧАЛЬНЫХ ДАННЫХ ИЛИ ВО ВРЕМЯ РАСЧЕТОВ
# подумать еще над случаем, когда выдает ответ в десятичной форме

In [32]:
answer_df_from_different_runs = []

In [33]:
for i in range(20):
    mistral_answers =[]
    for question in questions:
        
        answer = ask_question(question)
        mistral_answers.append(answer)
    answers_df = pd.DataFrame({'mistral_answers': mistral_answers, 'true_answers': short_answers, 'check': False})
    answers_df = answers_df.apply(check_answer, axis='columns')
    answer_df_from_different_runs.append(answers_df)

In [42]:
answer_df_from_different_runs[12].query('check==False').count()

mistral_answers    54
true_answers       54
check              54
dtype: int64

# Save DFs

In [43]:
import pickle

In [44]:
with open("baseline_dfs", "wb") as fp:   #Pickling
   pickle.dump(answer_df_from_different_runs, fp)

In [45]:
with open("baseline_dfs", "rb") as fp:   # Unpickling
   b = pickle.load(fp)

In [49]:
len(b)

20

# Прогон трех моделей с голосованием

In [11]:
def check_triple_answer(row):
    # добавить точки и запятые
    answer_length = len(row.true_answers)
    matches = [False, False, False]
    end_of_mistral_answer_1 = row.mistral_answers_1[-200:]
    end_of_mistral_answer_2 = row.mistral_answers_2[-200:]
    end_of_mistral_answer_3 = row.mistral_answers_3[-200:]
    if answer_length < 3:
        matches[0] = bool(re.search(r'\D' + str(row.true_answers) + r'\D', end_of_mistral_answer_1))
        matches[1] = bool(re.search(r'\D' + str(row.true_answers) + r'\D', end_of_mistral_answer_2))
        matches[2] = bool(re.search(r'\D' + str(row.true_answers) + r'\D', end_of_mistral_answer_3))
    # для случаев 4,351
    else:
        matches[0] = row.true_answers in end_of_mistral_answer_1 or row.true_answers[:-3] + ',' + row.true_answers[-3:] in end_of_mistral_answer_1 or row.true_answers[:-3] + '.' + row.true_answers[-3:] in end_of_mistral_answer_1
        matches[1] = row.true_answers in end_of_mistral_answer_2 or row.true_answers[:-3] + ',' + row.true_answers[-3:] in end_of_mistral_answer_2 or row.true_answers[:-3] + '.' + row.true_answers[-3:] in end_of_mistral_answer_2
        matches[2] = row.true_answers in end_of_mistral_answer_3 or row.true_answers[:-3] + ',' + row.true_answers[-3:] in end_of_mistral_answer_3 or row.true_answers[:-3] + '.' + row.true_answers[-3:] in end_of_mistral_answer_3

    row.check = True if matches.count(True) > 1 else False

    return row
# !!!!!!!!!!!!!!!!!!!
# ЕСТЬ РИСК ТОГО, ЧТО ОТВЕТ ВСТРЕЧАЕТСЯ В НАЧАЛЬНЫХ ДАННЫХ ИЛИ ВО ВРЕМЯ РАСЧЕТОВ
# подумать еще над случаем, когда выдает ответ в десятичной форме

In [8]:
answer_df_from_triple_different_runs = []

In [12]:
for i in range(20):
    triple_mistral_answers =[[], [], []]
    for question in questions:
        for i in range(3):
            answer = ask_question(question)
            triple_mistral_answers[i].append(answer)
    triple_answers_df = pd.DataFrame({'mistral_answers_1': triple_mistral_answers[0], 'mistral_answers_2': triple_mistral_answers[1], 
                                      'mistral_answers_3': triple_mistral_answers[2], 'true_answers': short_answers, 'check': False})
    triple_answers_df = triple_answers_df.apply(check_triple_answer, axis='columns')
    answer_df_from_triple_different_runs.append(triple_answers_df)

In [14]:
import pickle

In [15]:
with open("triple_dfs", "wb") as fp:   #Pickling
   pickle.dump(answer_df_from_triple_different_runs, fp)

In [17]:
answer_df_from_triple_different_runs[19]

,mistral_answers_1,mistral_answers_2,mistral_answers_3,true_answers,check
0,"In April, Natalia sold clips to 48 friends. I...","In April, Natalia sold clips to 48 friends. I...","In April, Natalia sold clips to 48 friends. I...",72,True
1,To find out how much Weng earned for babysitt...,To find out how much Weng earned for babysitt...,"First, let's convert the number of minutes in...",10,True
2,Betty currently has $15 from her parents. Her...,Let's calculate how much money Betty currentl...,"First, let's find out how much money Betty's ...",5,False
3,Let's break down the problem step by step:\n\...,Let's break down this problem step by step:\n...,Let's first find out how many pages Julie has...,42,True
4,James writes a 3-page letter to each of his t...,James writes a 3-page letter to each of his t...,James writes a 3-page letter to each of his t...,624,False
...,...,...,...,...,...
95,Let's first find out how many nickels Alice r...,"First, let's find out how many nickels Alice ...","First, let's find out how many nickels Alice ...",64,False
96,"First, let's calculate how many gallons of wa...","First, let's find out how many gallons of wat...","First, let's calculate how many gallons of wa...",126,True
97,Let's break down the number of books Nancy sh...,To find out how many books Nancy had on the c...,Let's break down the number of books Nancy sh...,46,False
98,Let's denote the number of gumballs Hector in...,Let's break down the problem by finding out h...,Let's denote the number of gumballs Hector or...,45,False
